In [ ]:
import numpy as np
from tqdm import tqdm
from itertools import product

In [ ]:
with open("input") as fp:
    *shapes, instr = fp.read().split("\n\n")

    instr = [x.split(": ") for x in instr.split("\n")]
    instr = [(*map(int, x[0].split("x")), *map(int, x[1].split())) for x in instr]

    shapes = [
        np.fromstring(
            s.split(":")[1].replace("\n", "").replace("#", "1,").replace(".", "0,"),
            sep=",",
            dtype=int,
        ).reshape(3, 3)
        for s in shapes
    ]

In [ ]:
def show(s):
    print("\n".join(map("".join, s.astype("str"))).replace("1", "#").replace("0", "."))


def aug_shape(s):
    return [np.rot90(s, k=k) for k in range(4)]


def test(shapes, x, left):
    if left.sum() == 0:
        return True

    x_rows, x_cols = x.shape
    n = 3

    for si in np.argwhere(left > 0).reshape(-1):
        for s in aug_shape(shapes[si]):
            # for i, j in np.argwhere(x == 0): # TODO: not sufficient to start from an empty cell in the grid
            for i, j in product(range(x_rows), range(x_cols)):
                if i <= x_rows - n and j <= x_cols - n:
                    y = x.copy()
                    y[i : i + n, j : j + n] += s
                    if y.max() > 1:  # overlap
                        continue
                    left[si] -= 1
                    # show(y); print(left); print('\n')
                    ok = test(shapes, y, left)
                    if ok:
                        break
                    else:
                        left[si] += 1

    return left.sum() == 0

In [ ]:
p1 = 0
for c, r, *task in instr:
    ok = sum(t * shapes[i].sum() for i, t in enumerate(task)) <= c * r
    p1 += ok
print(p1.item())